In [2]:
using LinearAlgebra,Random,Printf,LaTeXStrings
using Plots; gr()
using FFTW

In [3]:
function lattice(N::Int, D::Int) #construct array of lattice coordinates (with n = 0 in the center)
    @assert(N > 0, "N = " * string(N) * " must be positive")
    @assert(D > 0, "D = " * string(D) * " must be positive")
    
    if mod(N,2) == 0 n_1D = -(N-2)/2:N/2 end
    if mod(N,2) == 1 n_1D = -(N-1)/2:(N-1)/2 end

    if D == 1 return collect(n_1D) end
    
    n = collect(Base.product([n_1D for i in 1:D]...)) #Cartesian product
    
    return n
end

function potential(μ::Number, ϵ::Number, n::Array)
    @assert(μ > 0, "μ = " * string(μ) * " must be positive")
    @assert(ϵ > 0, "ϵ = " * string(ϵ) * " must be positive")
    
    return @. μ/8 * (ϵ^2*dot(n,n) - 1)^2
end

function kinetic(μ::Number, ϵ::Number, ψ::Array) #with periodic boundary conditions
    @assert(μ > 0, "μ = " * string(μ) * " must be positive")
    @assert(ϵ > 0, "ϵ = " * string(ϵ) * " must be positive")
    
    D = ndims(ψ)
    N = size(ψ)[1]

    Δ = -2*D*ψ #last term of discretized Laplacian

    for k in 1:D
        Δ += selectdim(ψ, k, vcat(2:N,1:1)) + selectdim(ψ, k, vcat(N:N,1:N-1)) #remaining terms
    end
    
    return -1/(2*μ*ϵ^2) * Δ
end

function Hamiltonian(μ::Number, ϵ::Number, ψ::Array, pot::Array) #it would likely be more memory-efficient to pre-calculate the potential
    
    return kinetic(μ, ϵ, ψ) + pot .* ψ
end

Hamiltonian (generic function with 1 method)

In [4]:
function power_method(vshape::Tuple{Vararg{Int}}, apply_A::Function, tol::Float64, maxiters::Int)
    @assert(tol > 0, "tol = " * string(tol) * " must be positive")
    @assert(maxiters > 0, "maxiters = " * string(maxiters) * " must be positive")
    
    v = rand(ComplexF64, vshape)
    niters = 0
    λ = 0
    
    while niters < maxiters
        w = apply_A(v)
        λ = norm(w)
        if max(abs.(w - λ*v)...) < tol break end
        v = w/λ
        niters += 1
    end
    
    if niters == maxiters error("Maximum number of power method iterations reached") end
    
    return λ, v, niters
end

function conjugate_gradient(apply_A::Function, b::Array, tol::Float64, maxiters::Int)
    @assert(tol > 0, "tol = " * string(tol) * " must be positive")
    @assert(maxiters > 0, "maxiters = " * string(maxiters) * " must be positive")
    
    x = rand(ComplexF64, size(b))
    r = b - apply_A(x); p = r
    niters = 0
    
    while niters < maxiters
        x = x + dot(r,r) / dot(p,apply_A(p)) * p
        r_new = b - apply_A(x)
        if norm(r_new) < tol*norm(b) break end
        p = r_new + dot(r_new,r_new) / dot(r,r) * p
        r = r_new
        niters += 1
    end
    
    if niters == maxiters error("Maximum number of conjugate gradient iterations reached") end
    
    return x, niters
end

function lowest_eigenvalue(vshape::Tuple{Vararg{Int}}, apply_A::Function, tol_pm::Float64, maxiters_pm::Int, tol_cg::Float64, maxiters_cg::Int)
    
    function apply_A_inverse(v)
        x, _ = conjugate_gradient(apply_A, v, tol_cg, maxiters_cg)
        return x
    end
    
    λ, v, _ = power_method(vshape, apply_A_inverse, tol_pm, maxiters_pm)
    
    return 1/λ, v
end

lowest_eigenvalue (generic function with 1 method)

In [5]:
function euler_integrator(Hamiltonian::Function, Potential::Array, τ::Float64, μ::Float64, ϵ::Float64, ψ::Array)
    dψ = ψ - im*Hamiltonian(μ, ϵ, ψ, Potential)*τ
    return dψ
end

function crank_nicolson(Hamiltonian::Function, Potential::Array, τ::Float64, μ::Float64, ϵ::Float64, ψ::Array, conj_grad::Function, tol::Float64, maxiters::Int)
    
    function apply_nicolson(ψ::Array)
        return ψ + 1/4*τ^2*Hamiltonian(μ, ϵ, Hamiltonian(μ, ϵ, ψ, Potential), Potential)
    end

    dψ, _ = conj_grad(apply_nicolson, ψ, tol, maxiters)
    Hdψ = Hamiltonian(μ, ϵ, dψ, Potential)
    dψ = dψ - im*τ*Hdψ - τ^2/4*Hamiltonian(μ, ϵ, Hdψ, Potential)
    
    return dψ
end

crank_nicolson (generic function with 1 method)

In [6]:
function gaussian(n::Array, σ::Number, n_0::Tuple, k_0::Tuple)
    x = [i .- n_0 for i in n]
    xk_0 = [dot(i,k_0) for i in n]
    return @. 1/sqrt(2π*σ^2)*exp(-dot(x,x)/(2*σ^2))*exp(im*xk_0)
end

n = lattice(200,2)
n_1D = lattice(200,1)
μ = 20.
τ = 0.004
T = 5
ϵ = sqrt(0.001)
#n = lattice(Int(round(3/ϵ)),1)
n_0 = (Int(round(1/(ϵ))),Int(round(1/(ϵ))))
k_0 = (0.27,0.27)
σ = 3
pot = potential(μ, ϵ, n)
pgaussian = gaussian(n, σ, n_0, k_0)
pegaussian = pgaussian/norm(pgaussian)
pcgaussian = pgaussian/norm(pgaussian)

anim = @animate for 🐒 in 1:Int(T/τ)
    #egaussian = euler_integrator(Hamiltonian, pot, τ, μ, ϵ, pegaussian/norm(pegaussian))
    cgaussian = crank_nicolson(Hamiltonian, pot, τ, μ, ϵ, pcgaussian, conjugate_gradient, 10e-4, 3000)
    global pcgaussian = cgaussian
    #global pegaussian = egaussian
    p = surface(n_1D, n_1D, abs.(cgaussian).^2, c = :blues, legend = nothing, camera = (180*log(🐒/(T/τ)+1), 50*log(🐒/(T/τ)+1)))
    surface!(n_1D, n_1D, pot/norm(pot), alpha = :0.2, c = :heat, legend = nothing)
    zlims!(0,0.01)
    #plot!(n, abs.(egaussian/norm(pegaussian)).^2, color = 1, label = "euler")
    #ylims!(-0.01,0.15)
end every 10

gif(anim, "quantum.gif", fps = 20)


In [1]:
n = lattice(200,1)
μ = 20.
ϵ = sqrt(0.001)
T = 5

n_0 = (Int(round(1/(ϵ))),)
k_0 = (0,)
σ = 3

τ = range(0.3, 0.0001, 50)
pot = potential(μ, ϵ, n)
anim = @animate for j ∈ τ
    pgaussian = gaussian(n, σ, n_0, k_0)
    pegaussian = pgaussian/norm(pgaussian)
    pcgaussian = pgaussian/norm(pgaussian)
    for i in 1:Int(round(T/j))
        cgaussian = crank_nicolson(Hamiltonian, pot, j, μ, ϵ, pcgaussian, conjugate_gradient, 10e-4, 3000)
        egaussian = euler_integrator(Hamiltonian, pot, j, μ, ϵ, pcgaussian)
        pcgaussian = cgaussian
        pegaussian = egaussian
    end
    p = plot(n, abs.(pcgaussian).^2, label = "Crank-Nicolson",legend=:topleft)
    plot!(n, abs.(pegaussian).^2, label = "Euler")
    title!("τ="*string(round(j,digits = 5))*", T="*string(T))
    ylims!(0,0.1)
end

gif(anim, "cool_limit.gif", fps = 5)

UndefVarError: UndefVarError: `lattice` not defined

In [ ]:
'anim = @animate for 🐒 in 1:Int(T/τ)
    #egaussian = euler_integrator(Hamiltonian, pot, τ, μ, ϵ, pegaussian/norm(pegaussian))
    cgaussian = crank_nicolson(Hamiltonian, pot, τ, μ, ϵ, pcgaussian, conjugate_gradient, 10e-4, 3000)
    global pcgaussian = cgaussian
    #global pegaussian = egaussian
    p = plot(n, pot/norm(pot), label = "potential", color = 3)
    plot!(n, abs.(cgaussian).^2, color = 2, label = "crank-nicolson")
    #plot!(n, abs.(egaussian/norm(pegaussian)).^2, color = 1, label = "euler")
    ylims!(-0.01,0.15)
end every 50'